## Importing packages

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import json
import math

#import geopandas as gpd

import pgeocode
from geopy.geocoders import Nominatim

import folium
import folium.plugins as plugins

from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, HeatMapWithTime


from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
import requests

## Cleaning Data (snapchat)

In [3]:
location = json.loads(open("data/location_history.json").read())
areas_visited = location['Areas you may have visited in the last two years']

location_list = list()

for j in range(len(areas_visited)):
    location_list.append([areas_visited[j]['Time'], areas_visited[j]['City'], areas_visited[j]['Region'], areas_visited[j]['Postal Code']])

location_df = pd.DataFrame (location_list, columns = ['Time', 'City','Region', 'Postal Code'])

location_df['loc'] = location_df['City'] + ', ' + location_df['Region']

geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
location = geolocator.geocode('4550 galloway, Ohio')
# print(location)
# print((location.latitude, location.longitude))

In [4]:
def loc_convert_lat(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location.latitude

def loc_convert_lang(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location.longitude

def loc_convert_address(x):
    
    geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")
    location = geolocator.geocode(x)
    return location

In [5]:
location_df['address'] = location_df['loc'].apply(loc_convert_address)
location_df['lat'] = location_df['loc'].apply(loc_convert_lat)
location_df['long'] = location_df['loc'].apply(loc_convert_lang)
location_df

,Time,City,Region,Postal Code,loc,address,lat,long
0,2021/02/14 16:16:02 UTC,galloway,Ohio,43119,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
1,2021/02/12 03:38:56 UTC,galloway,Ohio,48104,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
2,2021/01/31 15:13:16 UTC,galloway,Ohio,48197,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
3,2021/01/31 04:01:26 UTC,galloway,Ohio,48212,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
4,2020/12/24 20:38:21 UTC,galloway,Ohio,48242,"galloway, Ohio","(Galloway, Prairie Township, Franklin County, ...",39.913921,-83.163334
...,...,...,...,...,...,...,...,...
113,2020/04/20 04:02:37 UTC,dayton,Ohio,48109,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607
114,2020/03/02 20:30:43 UTC,dayton,Ohio,48212,"dayton, Ohio","(Dayton, Montgomery County, Ohio, 45423, Unite...",39.758948,-84.191607
115,2021/08/13 19:03:31 UTC,battle creek,Michigan,48104,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427
116,2021/05/17 03:16:52 UTC,battle creek,Michigan,48212,"battle creek, Michigan","(Battle Creek, Calhoun County, Michigan, 49014...",42.319255,-85.182427


## Timelapse Map

In [9]:
selected_columns = location_df[["Time", "lat", "long"]]
timelapse_df = selected_columns.copy()
#timelapse_df

lat_long_list = []
for i in timelapse_df['Time'].unique():
    temp=[]
    for index, instance in timelapse_df[timelapse_df['Time'] == i].iterrows():
        temp.append([instance['lat'],instance['long']])
    lat_long_list.append(temp)
    
#converting it to datetime format
timelapse_df['Time']= pd.to_datetime(timelapse_df['Time'])
#creating a time index
time_index = []
for i in timelapse_df['Time'].unique():
    time_index.append(i)
#formatting the index
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]


# -------------------------------------------------------------------------------- #

#Choosing the map type 
timelapse_map = folium.Map(location=[42.2808, -83.7430],zoom_start = 5, tiles="openstreetmap",attr="Stadia.AlidadeSmoothDark")
#Plot it on the map
HeatMapWithTime(lat_long_list,radius=10,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(timelapse_map)
# Display the map

# Adds tool to the top right
from folium.plugins import MeasureControl
timelapse_map.add_child(MeasureControl())

timelapse_map

## Cluster Map

In [11]:
# Create the map
cluster_map = folium.Map(location=[42.2808, -83.7430], tiles='openstreetmap', zoom_start=5)

# Add points to the map
mc = MarkerCluster()
for idx, row in location_df.iterrows():
    if not math.isnan(row['long']) and not math.isnan(row['lat']):
        mc.add_child(Marker([row['lat'], row['long']]))
cluster_map.add_child(mc)

# Adds tool to the top right
from folium.plugins import MeasureControl
cluster_map.add_child(MeasureControl())

# Display the map
cluster_map

## Foursquare code w/o specific query

In [13]:
CLIENT_ID = 'PDLBYPAUCNCTOCDJ0MHINYM5X2MM5QLJUNBVV2JUNQPKWYPW' # your Foursquare ID
CLIENT_SECRET = 'FEI04Q3MFOROEJWTFW20V0ZOFVYOLLXZIZUJCLR1R2TCL3TU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

In [16]:
ndf = location_df[['loc','lat','long']]
nndf = ndf.drop_duplicates()
nndf["long_lat"] = list(zip(nndf["lat"], nndf["long"]))

concat_df = pd.DataFrame()

for index, row in nndf.iterrows():
    url = "https://api.foursquare.com/v3/places/nearby?ll={},{}&limit=5".format(round(row['lat'], 2),round(row['long'], 2))
#     print(url)
    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3chVTJib0Z11IA8qFisvs8p7dkOJ6ky0WEbGTZ9FQPqc="
    }
    result = requests.request("GET", url, headers=headers).json()
    result_df = json_normalize(result['results'])
    result_df = result_df.rename(columns={"geocodes.main.latitude": "lat", "geocodes.main.longitude": "long", "location.formatted_address": "address"})
#     print(result_df)
    concat_df = concat_df.append(result_df)
    
    
# -------------------------------------------------------------------------------- #

    
foursquare_map_no_query = folium.Map(location=[42.2808, -83.7430], zoom_start=7)
# add a red circle marker to represent each visited locations
for lat, long in zip(nndf.lat, nndf.long):
    folium.features.CircleMarker(
        [lat, long],
        radius=10,
        color='red',
        #popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(foursquare_map_no_query)
# add all venues as blue circle markers
for lat, long, label in zip(concat_df.lat, concat_df.long, concat_df.name):
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(foursquare_map_no_query)
# Adds tool to the top right
from folium.plugins import MeasureControl
foursquare_map_no_query.add_child(MeasureControl())

<ipython-input-16-120864580874>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nndf["long_lat"] = list(zip(nndf["lat"], nndf["long"]))
<ipython-input-16-120864580874>:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  result_df = json_normalize(result['results'])


In [22]:
concat_df = concat_df[['name', 'distance']]
concat_df

,name,distance
0,Mentel Memorial Golf Course,790
1,Health Care Logistics,454
2,Galloway Cemetery,252
3,Galloway Tavern,486
4,Saia,2281
...,...,...
0,St Philip Catholic Church,108
1,Battle Creek Public Schools,129
2,Clara's on the River,171
3,BP,8354


In [24]:
print('Red dots represent each visited locations and Blue dots represent surrounding popular venues')

Red dots represent each visited locations and Blue dots represent surrounding popular venues


## Foursquare code w/ specific query

In [25]:
user_input = input()

indian


In [27]:
concat_df2 = pd.DataFrame()
for index, row in nndf.iterrows():
    url = "https://api.foursquare.com/v3/places/search?query={}&ll={},{}&radius=50000&limit=5".format(user_input, row['lat'], row['long'])
#     print(url)
    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3chVTJib0Z11IA8qFisvs8p7dkOJ6ky0WEbGTZ9FQPqc="
    }
    result = requests.request("GET", url, headers=headers).json()
    result_df = json_normalize(result['results'])
    result_df = result_df.rename(columns={"geocodes.main.latitude": "lat", "geocodes.main.longitude": "long", "location.formatted_address": "address"})
#     print(result_df)
    concat_df2 = concat_df2.append(result_df)
    
concat_df2 = concat_df2.dropna(subset=['lat', 'long', 'name'])


# -------------------------------------------------------------------------------- #


foursquare_map_with_query = folium.Map(location=[42.2808, -83.7430], zoom_start=7)
# add a red circle marker to represent each visited locations
for lat, long in zip(nndf.lat, nndf.long):
    folium.features.CircleMarker(
        [lat, long],
        radius=10,
        color='red',
        #popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(foursquare_map_with_query)
# # add all venues as blue circle markers
for lat, long, label in zip(concat_df2.lat, concat_df2.long, concat_df2.name):
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(foursquare_map_with_query)
    
foursquare_map_with_query

<ipython-input-27-6fe537569a92>:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  result_df = json_normalize(result['results'])


In [28]:
concat_df2 = concat_df2[['name', 'distance']]
concat_df2

,name,distance
0,Aab India Restaurant,13032
1,Amul India,20108
2,Indian Oven,15709
3,Flavors of India,14997
4,Curry & Hurry Indian Restaurant,16959
...,...,...
0,Saffron Indian Cuisine,35194
1,Cumin Indian Cuisine,37305
2,Indian Cuisine,35768
3,The Coop,33854


In [29]:
print('Red dots represent each visited locations and Blue dots represent surrounding popular venues')

Red dots represent each visited locations and Blue dots represent surrounding popular venues


## Popular Venues at Each Location Table

In [31]:
sumdf = location_df[['loc','lat','long']]
sumdf = ndf.drop_duplicates()

new = concat_df['name'].to_frame()

venue1 = new.loc[0]['name'].to_list()
venue2 = new.loc[1]['name'].to_list()
venue3 = new.loc[2]['name'].to_list()

sumdf["1st most Popular Venue"] = venue1
sumdf["2nd most Popular Venue"] = venue2
sumdf["3rd most Popular Venue"] = venue3

sumdf = sumdf.reset_index(drop=True)
sumdf

<ipython-input-31-05ea92f8a3b9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumdf["1st most Popular Venue"] = venue1
<ipython-input-31-05ea92f8a3b9>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumdf["2nd most Popular Venue"] = venue2
<ipython-input-31-05ea92f8a3b9>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

,loc,lat,long,1st most Popular Venue,2nd most Popular Venue,3rd most Popular Venue
0,"galloway, Ohio",39.913921,-83.163334,Mentel Memorial Golf Course,Health Care Logistics,Galloway Cemetery
1,"ann arbor, Michigan",42.268157,-83.731229,Delta Kappa Epsilon,The Metal Frat,Floor Trader
2,"chicago, Illinois",41.875562,-87.624421,Grant Park - Butler Field,Millennium Lakeside Garage,The Modern Wing
3,"brighton, Michigan",42.531692,-83.784646,Sushi Zen,CVS Pharmacy,Thai Express
4,"columbus, Ohio",39.962260,-83.000707,CVS Pharmacy,Tasty Dawg,Peanut Shoppe
5,"toledo, Ohio",41.652914,-83.537817,M' Osteria and Bar,Blarney Irish Pub,Chevy's Place
6,"akron, Ohio",41.083064,-81.518485,Peanut Shoppe,The Lockview,Lock 3 Park
7,"detroit, Michigan",42.331551,-83.046640,Theodore Levin United States Courthouse,PenzDetroiT,Gateway Deli Cafe
8,"elberton, Georgia",34.111223,-82.867084,Elbert County Granite Bowl,United States Postal Service,The White Plate Restaurant
9,"dearborn, Michigan",42.322260,-83.176315,Kidcadia,J B Bamboozle's,Good Burger East Dearborn
